In [ ]:
import sys,os
sys.path.append('..')
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.swarms.manager import SwarmManager
from backtester.exoinfo import EXOInfo

import pandas as pd
import numpy as np
import scipy

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
swarms = {}
for file in os.listdir('./swarms/'):
    swarm_name = file.replace('.swm','')
    swarms[swarm_name] = SwarmManager.load(os.path.join('swarms', file))

In [ ]:
swarms

In [ ]:
passive_exo = {}

exos_mat_files = [
    './exo/strategy_2010348.mat',
    './exo/strategy_2060248.mat',    
]

print('Passive EXO list')
for matfile in exos_mat_files:    
    exo = EXOInfo.from_matfile(matfile)
    passive_exo[exo.exo_name()] = exo.exo_price_index()
    
    print(exo.exo_name())

In [ ]:
exo.data.fut_price.plot()

In [ ]:

campaign_exposure = {
     'ZLE_BearishCollar_MACross_Long': 1,
     'ZLE_BearishCollar_MACross_Short': 1,
     'ZLE_BearishCollar_SwingPoint_Short': 1,
     'ZLE_BearishPut_MACross_Long': 1,
     'ZLE_BearishPut_MACross_Short': 1,
     'ZLE_BullishCall_MACross_Long': 1,
     'ZLE_BullishCall_MACross_Short': 1,
     'ZLE_BullishCollar_MACross_Long': 1,
     'ZLE_BullishCollar_MACross_Short': 1,
     #
     # Passive EXO positions
     #
     'ZLE_BullishCall_EXO': -1, # Short 1 EXO units
     'ZLE_BearishCollar_EXO': 1, # Long 1 EXO units
     
}

In [ ]:
#def campaign_equity(swarms, exposure):

campaign_dict = {}

for camp_name, exposure in campaign_exposure.items():
    if camp_name in swarms:
        swarm_name = camp_name
        smgr = swarms[swarm_name]
        campaign_dict[swarm_name] = smgr.swarm_picked.diff().sum(axis=1) * exposure
    elif camp_name in passive_exo:
        exo_equity = passive_exo[camp_name]
        campaign_dict[camp_name] = exo_equity.diff() * exposure
    
    
campaign_portfolio = pd.DataFrame(campaign_dict).cumsum()
campaign_equity = campaign_portfolio.sum(axis=1)

In [ ]:
figsize(10,10)
campaign_portfolio.plot()

In [ ]:
campaign_portfolio.sum(axis=1).plot()

## Campaign results

In [ ]:
size_futures_long = 80
size_futures_short = -80
size_campaign = 40

In [ ]:
futures_index = exo.data.fut_price.diff().cumsum()

## long/short futures

In [ ]:
(futures_index*-1*size_futures_short).plot(label='Short futures')
(futures_index*size_futures_long).plot(label='Long futures')

legend(loc=2)

## Futures and bidirectional campaign

In [ ]:
(futures_index*size_futures_short).plot(label='Short futures')
(futures_index*size_futures_long).plot(label='Long futures')
(campaign_equity*size_campaign).plot(label='Campaign')

legend(loc=2)

In [ ]:
results_dict = {
    'long_futures_equity': futures_index*size_futures_long,
    'short_futures_equity': futures_index*size_futures_short,
    'campaign_equity': campaign_equity*size_campaign
}

pd.DataFrame(results_dict).to_csv('./results/campaign_results.csv')


In [ ]:
import json
campaign_info = {
    'structure': campaign_exposure,
    'sizes': {
        'long_futures': size_futures_long,
        'short_futures': size_futures_short,
        'campaign_units': size_campaign
    }
}

with open('./results/campaign_info.json', 'w') as f:
    json.dump(campaign_info, f, indent=4)